In [86]:
import os 
import sys
import xarray as xr
import numpy as np
import pandas as pd

import cartopy.crs as crs
import cmocean
import seaborn as sns
import hvplot.pandas
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
import colorcet as cc
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from pycoawst.tools.grid import cart2polar

home = os.environ['HOME']
pd.options.plotting.backend = 'holoviews'
xr.set_options(display_style ="html")

In [2]:
from dask.distributed import Client
Client()

Client Scheduler: tcp://127.0.0.1:62412 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.00 GiB


In [27]:
# swname = home + "/Dropbox/Oceanography/Projects/annulus/floats/10channelfloats/annFloats_sw10.nc"
# nmname = home + "/Dropbox/Oceanography/Projects/annulus/floats/10channelfloats/annFloats_nm10.nc"
swname = home + "/Dropbox/Oceanography/Projects/annulus/floats/annfloat_sw2D.nc"
nmname = home + "/Dropbox/Oceanography/Projects/annulus/floats/annfloat_nm2D.nc"
gridname = home + "/Dropbox/Oceanography/Projects/annulus/grid/ann_grid.nc"

Dg = xr.open_dataset(gridname, decode_times = False)
Dp_sw = xr.open_dataset(swname, decode_times = False).isel(traj = 45).isel(obs = range(42,550)) #sees all subdomains
Dp_nm = xr.open_dataset(nmname, decode_times = False).isel(traj = 45).isel(obs = range(42,550)) #sees all subdomains
# Dp_sw = xr.open_dataset(swname, decode_times = False)#.isel(traj = 9)
# Dp_nm = xr.open_dataset(nmname, decode_times = False)#.isel(traj = 9)

Dp_sw['days'] = Dp_sw.time/86400
Dp_sw = Dp_sw.assign_coords(coords = {'time': Dp_sw.time, 'days': Dp_sw.days})
Dp_nm['days'] = Dp_nm.time/86400
Dp_nm = Dp_nm.assign_coords(coords = {'time': Dp_nm.time, 'days': Dp_nm.days})

#Dp_sw
#Dp_nm

In [28]:
df_sw = Dp_sw.to_dataframe()
# df_sw.lon[df_sw.lon>360] -= 360
# df_sw.lon -= 180
# df_sw['x'] = (90 - df_sw.lat)*np.cos(df_sw.lon*np.pi/180) 
# df_sw['y'] = (90 - df_sw.lat)*np.sin(df_sw.lon*np.pi/180)
# df_sw['r'] = np.hypot(df_sw.x, df_sw.y)

df_nm = Dp_nm.to_dataframe()
# df_nm.lon[df_nm.lon>360] -= 360
# df_nm.lon -= 180
# df_nm['x'] = (90 - df_nm.lat)*np.cos(df_nm.lon*np.pi/180) 
# df_nm['y'] = (90 - df_nm.lat)*np.sin(df_nm.lon*np.pi/180)
# df_nm['r'] = np.hypot(df_nm.x, df_nm.y)

lon_rho, lat_rho = cart2polar(Dg.x_rho, Dg.y_rho)

In [81]:
%matplotlib ipympl
proj = crs.NorthPolarStereo()
facecolor = 'w'
with sns.axes_style("white", rc={'text.usetex': True}):
   
    fig, ax = plt.subplots(1, 1, subplot_kw=dict(projection=proj), figsize = (8,6) ) 
    ax.background_patch.set_facecolor(facecolor)
    
    ax.set_extent([130,135,89.75,90], crs=crs.PlateCarree())
    #ax.set_extent([370, 420, 89.75, 89.9], crs=crs.PlateCarree())
    
    pc = ax.pcolormesh(lon_rho, lat_rho, Dg.h, transform = crs.PlateCarree(), cmap = cmocean.cm.ice_r, vmin = 1e-1 , vmax = 1e3,
                   edgecolor = [0,0,0,.25], linewidth = 0,  norm = LogNorm(), rasterized = True )

    s = ax.scatter(df_sw.lon, df_sw.lat, s = 2, c = df_sw.time/3600/24, transform = crs.PlateCarree(), cmap = cc.cm.fire, vmin = 14, vmax = 15.75)
   # s = ax.scatter(df_sw.lon, df_sw.lat, s = 2, c = df_sw.time/3600/24, transform = crs.PlateCarree(), cmap = cc.cm.fire, vmin = 14, vmax = 15)
    
    divider = make_axes_locatable(ax)
    ax_cb = divider.new_horizontal(size="5%", pad=0, axes_class=plt.Axes)
    fig.add_axes(ax_cb)
    
    ticks = [14.00, 14.25, 14.50, 14.75, 15.0, 15.25, 15.50, 15.75]
    cb = plt.colorbar(s, ax = ax, cax = ax_cb, pad = 0, ticks = ticks)#, orientation = 'horizontal')
    cb.set_label(r'Time (days)', color = 'k', size = 10)
    cb.ax.tick_params(labelsize=8, width = .5, length = 4)

    gl = ax.gridlines(linewidth = 0.25,  alpha = 0.5, color = 'gray')
    gl.n_steps = 360

    #plt.savefig("track.svg", dpi = 300, facecolor = facecolor, bbox_inches='tight')
    plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-81-c49c15768678>:7: DeprecationWarning: The background_patch property is deprecated. Use GeoAxes.patch instead.
  ax.background_patch.set_facecolor(facecolor)
/Users/WalterTorres/miniconda3/envs/viz/lib/python3.9/site-packages/cartopy/mpl/geoaxes.py:1702: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  X, Y, C, shading = self._pcolorargs('pcolormesh', *args,


### white background


In [29]:
#preprocessing

def ds2df(ds):
    ds["coriolis_bar"] = ds.cor_bar + ds.fsco_bar
    ds["vf_bar"] = ds.hjvf_bar + ds.kvrf_bar  
    ds["accel_bar"] = -ds.accel_bar
    
    return ds.to_dataframe()

df_sw = ds2df(Dp_sw) #Dp_sw.to_dataframe() #.isel(traj = i).to_dataframe()
df_nm = ds2df(Dp_nm) #Dp_nm.to_dataframe() #.isel(traj = i).to_dataframe()

In [96]:
    (
    (
    df_sw.plot(x = "days", y = "prsgrd_bar", color = "orange", **lineopts)*
    df_sw.plot(x = "days", y = "hadv_bar", color = "green", **lineopts)*
    df_sw.plot(x = "days", y = "bstr_bar", color = "purple", **lineopts)
    )
    +
    (
    df_nm.plot(x = "days", y = "prsgrd_bar", color = "orange",  **lineopts)*
    df_nm.plot(x = "days", y = "hadv_bar", color = "green", ax = ax[1], **lineopts)*
    df_nm.plot(x = "days", y = "coriolis_bar", color = "purple", ax = ax[1], **lineopts)
    )
    )

:Layout
   .Overlay.I  :Overlay
      .Curve.I   :Curve   [days]   (prsgrd_bar)
      .Curve.II  :Curve   [days]   (hadv_bar)
      .Curve.III :Curve   [days]   (bstr_bar)
   .Overlay.II :Overlay
      .Curve.I   :Curve   [days]   (prsgrd_bar)
      .Curve.II  :Curve   [days]   (hadv_bar)
      .Curve.III :Curve   [days]   (coriolis_bar)

In [113]:
from colorcet import glasbey_dark as glasbey

pd.options.plotting.backend = 'matplotlib'

#varlist = ["accel", "cor", "hadv", "fsco", "hjvf", "hvisc", "kvrf", "prsgrd", "vadv", "vjvf", "vvisc", "wbrk"]
#varlist = ["accel", "hadv", "vadv", "prsgrd", "coriolis", "hvisc", "vvisc", "vf", "wbrk"]

varlist = ["accel_bar", "hadv_bar", "prsgrd_bar", "coriolis_bar", "hvisc_bar", "bstr_bar", "vf_bar", "wbrk_bar"]
n = len(varlist)
ind = 85 #14,16 ok, 33 best, 35+43+68 good, 48+67+83+84 quite good, 86 best?


# rcparams = {'text.usetex': False, 
#             'text.latex.preamble' : [r'\usepackage{lmodern}']}
plt.rcParams.update({"text.usetex": False})


sns.set_palette("tab10")
# for i in range(10):

lineopts = {"line_width": 1,
            "ylim": (-1e-3,1e-3)}

with sns.axes_style("white", rc=rcparams):

    fig, ax = plt.subplots(3,1, figsize = (8, 10.5), sharex = True, facecolor = 'w')
    for axis in ax.flatten()[:-1]:
        axis.set_facecolor = 'w'
        axis.set_yscale('symlog', linthresh=5e-6)
        #axis.set_prop_cycle(color = glasbey[ind:ind+n])

        axis.set_ylabel('$ms^{-2}$', fontsize = 12)
        axis.spines['bottom'].set_visible(False)
        axis.spines['right'].set_visible(False)
        axis.spines['top'].set_visible(False)
        axis.yaxis.set_ticks_position('left')
        axis.yaxis.set_tick_params(labelsize = 8, width = .25)
        axis.xaxis.grid(True)
        #axis.xaxis.set_ticks_position('bottom')
        #axis.tick_params(direction='out', length=6, width=2, colors='r',
        #       grid_color='r', grid_alpha=0.5)

    ax[2].xaxis.set_ticks_position('bottom')
    ax[2].xaxis.set_tick_params(labelsize = 8, width = .25)
    ax[2].yaxis.set_ticks_position('left')
    ax[2].yaxis.set_tick_params(labelsize = 8, width = .25)
    ax[2].set_facecolor = 'w'
    ax[2].spines['right'].set_visible(False)
    ax[2].spines['top'].set_visible(False)

    df_sw.plot(x = "days", y = varlist, ax = ax[0], ylim = (-1e-3, 1e-3), linewidth = 1)
    df_nm.plot(x = "days", y = varlist, ax = ax[1], ylim = (-1e-3, 1e-3), linewidth = 1)
    df_sw["depth"] = -df_sw.h
    df_sw.plot(x = "days", y = "depth", ax = ax[2], linewidth = .75, linestyle = "--", color = "k")
    

    #ax[1].tick_params(width = 2, length = 5)

    ax[0].legend().set_visible(False)
    legend = ax[1].legend(frameon = False, borderaxespad=-12, prop={'size': 8}, bbox_to_anchor = (1.1,.4), )
    #legend.get_texts()[0].set_text('acceleration')
#     legend.get_texts()[1].set_text('horizontal advection')
#     legend.get_texts()[0].set_text('pressure gradient')
#     legend.get_texts()[2].set_text('bottom stress/coriolis')
#     legend.get_texts()[4].set_text('horizontal viscosity')
#     legend.get_texts()[5].set_text('bottom stress')
#     legend.get_texts()[6].set_text('vortex forces')
#     legend.get_texts()[7].set_text('wave breaking')
    
    #legend.get_texts()[6].set_text('$\it{k}$ vortex force')
    #legend.get_texts()[3].set_text('stokes coriolis')
    #legend.get_texts()[4].set_text('horizontal $\it{j}$ vortex force')
    #legend.get_texts()[9].set_text('vertical $\it{j}$ vortex force')
    

    #ax[2].set_xlim(14,15.85)

    ax[2].set_ylim(-150, 0)
    ax[2].set_ylabel("Depth (m)", fontsize = 12)
    ax[2].set_xlabel("Days", fontsize = 12)
    fig.subplots_adjust(hspace=0)
    plt.tight_layout()
    
    #plt.savefig('lagrangeMomentum-lite.svg',  dpi = 600, bbox_inches='tight') 
    #plt.savefig("{}.png".format(i),  dpi = 300, bbox_inches='tight')
    #plt.close(fig)
    
    #plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### dark background


In [ ]:
from colorcet import glasbey_light as glasbey
varlist = ["accel", "cor", "hadv", "fsco", "hjvf", "hvisc", "kvrf", "prsgrd", "vadv", "vjvf", "vvisc", "wbrk"]
n = len(varlist)
fig, ax = plt.subplots(3,1, figsize = (10, 8), sharex = True)
ind = 79 #60, 70 good, 8 glassby cool
ax[0].set_prop_cycle(color = glasbey[ind:ind+n])
ax[1].set_prop_cycle(color = glasbey[ind:ind+n])
ax[0].set_ylabel('$ms^{-2}$')
ax[1].set_ylabel('$ms^{-2}$')
ax[0].spines['bottom'].set_visible(False)
ax[1].spines['bottom'].set_visible(False)

df_sw.plot(x = "days", y = varlist, ax = ax[0], ylim = (-1e-5, 1e-5), linewidth = .75)
df_nm.plot(x = "days", y = varlist, ax = ax[1], ylim = (-1e-5, 1e-5), linewidth = .75)

ax[0].legend().set_visible(False)
legend = ax[1].legend(bbox_to_anchor = (1.1,1.5), frameon = False, loc = "center right")
plt.setp(legend.get_texts(), color = "w")

ax[2].plot(df_sw.days.values, -df_sw.h.values, 'w--', linewidth = .75)
ax[2].set_ylim(-150, 0)
ax[2].set_ylabel("Depth (m)")
ax[2].set_xlabel("Days")

# axh = ax[1].twinx()
# axh.set_ylabel('Depth (m)')
# axh.set_ylim()
# axh.plot(df_nm.days.values, -df_nm.h.values, 'w--')
plt.show()